In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import scipy.stats as stat
import pylab 
import math
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as LGR,Lasso
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

In [2]:
Fe_CAAC_dataset = df = pd.read_excel("F:\\Nitrogen_activation\\rajaraman\\office\\ofifice\\jupyter\\Extension\\T07\\heatmap\\agostic\\xgboost\\N2_activation_extension_T07_heatmap_agostaic_descrptors.xlsx")

In [3]:
df_1=df.drop(['F00','T01','T02','T03',],axis=1)
df_2=df_1.drop([0,18,9])
df_3=df_1.drop([0,18,9,8,26,17])
df_4=df_1.drop([8,26,17])

In [4]:
X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=20)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled_MM = scaler.transform(X_train)
X_test_scaled_MM = scaler.transform(X_test)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_train.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_test.columns)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


RF_reg_model = RandomForestRegressor()
# Define the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],  # Explore a range of depths
    'n_estimators': [100, 200],  # Example for number of trees
    'min_samples_split': [2, 5, 10],  # Control overfitting by setting min samples for split
    'min_samples_leaf': [1, 2, 4],  # Control overfitting by setting min samples per leaf
}

# Initialize the model
#rf = RandomForestRegressor(random_state=42)
#rf = RandomForestRegressor

# Set up Grid Search
grid_search = GridSearchCV(estimator=RF_reg_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
#grid_search.fit(X_train, y_train)

grid_search.fit(X_train_scaled_MM,y_train)

# Best parameters found
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 198 candidates, totalling 990 fits
Best parameters found:  {'max_depth': 90, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best score:  -264.1792106120085


In [16]:
RF_reg_model = RandomForestRegressor(max_depth=2, random_state=None,max_features='auto',n_estimators=200)

RF_reg_model.fit(X_train_scaled_MM,y_train)


# prediction on Training data
training_data_prediction_X_train_RF_reg = RF_reg_model.predict(X_train_scaled_MM)
# prediction on Test data
test_data_prediction_X_test_RF_reg = RF_reg_model.predict(X_test_scaled_MM)

# R squared Error
error_score_r2 = metrics.r2_score(y_train, training_data_prediction_X_train_RF_reg)
error_score_MAE = metrics.mean_absolute_error(y_train, training_data_prediction_X_train_RF_reg)
error_score_MSE = metrics.mean_squared_error(y_train, training_data_prediction_X_train_RF_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_train,training_data_prediction_X_train_RF_reg)))
# R squared Error
error_score_r2 = metrics.r2_score(y_test, test_data_prediction_X_test_RF_reg)
error_score_MAE = metrics.mean_absolute_error(y_test, test_data_prediction_X_test_RF_reg)
error_score_MSE = metrics.mean_squared_error(y_test, test_data_prediction_X_test_RF_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_test,test_data_prediction_X_test_RF_reg)))


R squared Error :  0.9475675327531967
mean_absolute_error :  6.714183297039192
mean_squared_error :  91.68438658224048
RMSE 9.575196425256271
R squared Error :  0.8664959892942745
mean_absolute_error :  8.592633001511894
mean_squared_error :  94.52129432144741
RMSE 9.722206247629568


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [17]:
# can tune other metrics, such as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X)
# transform train and test sets
X_scaled_MM = scaler.transform(X)
X_scaled_MM = pd.DataFrame(X_scaled_MM, columns=X.columns)

scores = cross_val_score(RF_reg_model, X_scaled_MM, y, cv=2)
print(scores)
print(np.mean(scores))

scores = cross_val_score(RF_reg_model, X_scaled_MM, y, cv=3)
print(scores)
print(np.mean(scores))

scores = cross_val_score(RF_reg_model, X_scaled_MM, y, cv=4)
print(scores)
print(np.mean(scores))

scores = cross_val_score(RF_reg_model, X_scaled_MM, y, cv=5)
print(scores)
print(np.mean(scores))




c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To k

[0.79859854 0.78528578]
0.7919421594448703


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[0.85934425 0.84406291 0.82428995]
0.8425657035670823


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To k

[0.73993187 0.878908   0.92320198 0.7834086 ]
0.8313626129831118


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To k

[0.56291563 0.92582297 0.85622681 0.90834702 0.69826908]
0.7903163018521774


In [21]:

# Initialize the Leave-One-Out cross-validator
loo = LeaveOneOut()

X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model


# Apply MinMax Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Scale the features and store them in X_scaled


# Initialize lists to store predictions and actual values
y_pred_loo = []
y_true_loo = []

# Loop over each training/test split
for train_index, test_index in loo.split(X_scaled):
    X_train_loo, X_test_loo = X_scaled[train_index], X_scaled[test_index]
    y_train_loo, y_test_loo = y.iloc[train_index], y.iloc[test_index]
    
    # Train the Lasso model
RF_reg_model = RandomForestRegressor(max_depth=2, random_state=None,max_features='auto',n_estimators=200)
RF_reg_model.fit(X_train_loo, y_train_loo)
    
# Predict the left-out sample
y_pred_loo.append(RF_reg_model.predict(X_test_loo)[0])
y_true_loo.append(y_test_loo.values[0])

# Convert lists to arrays
y_pred_loo = np.array(y_pred_loo)
y_true_loo = np.array(y_true_loo)

# Calculate LOO performance metrics
Q2_loo = r2_score(y_true_loo, y_pred_loo)
MAE_loo = mean_absolute_error(y_true_loo, y_pred_loo)
MSE_loo = mean_squared_error(y_true_loo, y_pred_loo)
RMSE_loo = np.sqrt(MSE_loo)

# Print the results
print("LOO Q^2 score: ", Q2_loo)
print("LOO Mean Absolute Error: ", MAE_loo)
print("LOO Mean Squared Error: ", MSE_loo)
print("LOO Root Mean Squared Error: ", RMSE_loo)



LOO Q^2 score:  nan
LOO Mean Absolute Error:  3.8029839480878422
LOO Mean Squared Error:  14.462686909413792
LOO Root Mean Squared Error:  3.8029839480878422


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
